In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [2]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [3]:
# !pip install git+https://github.com/huggingface/transformers

In [1]:
!ls -lah /root/.cache/huggingface/hub/

total 81M
drwxr-xr-x 10 root root 4.0K Oct 26 13:21 .
drwxr-xr-x  5 root root   61 Oct 24 15:26 ..
drwxr-xr-x  6 root root   65 Aug 21 09:16 models--bigscience--bloom-1b7
drwxr-xr-x  6 root root   65 Oct 24 18:26 models--bigscience--bloom-7b1
drwxr-xr-x  6 root root   65 Aug 30 22:52 models--FlagAlpha--Llama2-Chinese-7b-Chat
drwxr-xr-x  6 root root   65 Oct 24 11:46 models--meta-llama--Llama-2-7b-chat-hf
drwxr-xr-x  6 root root   65 Sep  5 00:14 models--shibing624--text2vec-base-chinese
drwxr-xr-x  2 root root    6 Aug 29 13:57 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x  6 root root   65 Oct 26 13:21 models--tiiuae--falcon-7b
drwxr-xr-x  5 root root   48 Oct 26 09:36 models--ysw96--my_awesome_peft_model
-rw-------  1 root root  80M Oct 26 08:22 tmpd_8zze2t
-rw-------  1 root root    0 Oct 24 11:18 tmpxe5e_q0g
-rw-r--r--  1 root root    1 Jul 26 16:10 version.txt


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [6]:
!pip show transformers

Name: transformers
Version: 4.35.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [4]:
!pip show peft

Name: peft
Version: 0.6.0.dev0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/site-packages
Requires: accelerate, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [3]:
!nvidia-smi

Mon Oct 30 16:46:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   28C    P0              68W / 300W |  19484MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq bitsandbytes==0.39.0 --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
# !pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
# !pip install -qqq datasets==2.12.0 --progress-bar off
# !pip install -qqq loralib==0.1.1 --progress-bar off
# !pip install -qqq einops==0.6.1 --progress-bar off

In [5]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-10-30 16:46:48.014019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 16:46:48.814501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import torch
torch.cuda.device_count()

1

In [7]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [8]:
MODEL_NAME = "FlagAlpha/Llama2-Chinese-7b-Chat"
# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # trust_remote_code=True,
    local_files_only=True,
    quantization_config=bnb_config,
    # temperature=0.0
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [9]:
!nvidia-smi

Mon Oct 30 16:49:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   28C    P0              68W / 300W |  23763MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [11]:
print_trainable_parameters(model)

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


In [12]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
# config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["query_key_value"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


## Inference Before Training

In [14]:
prompt = f"""
: 权益购买的功能用途是什么？
:
""".strip()
print(prompt)

: 权益购买的功能用途是什么？
:


In [17]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.0
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [18]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.0,
  "top_p": 0.7
}

In [19]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: 权益购买的功能用途是什么？
: 权益购买是一种在互联网上购买权益的方式，通常是指在社交媒体上购买的权益。权益购买的功能用途是为了让用户在社交媒体上获得更多的权益，例如在Facebook上购买的权益可以为用户提供更多的社交功能，例如更多的朋友、更多的粉丝、更多的赞评等。

CPU times: user 16.2 s, sys: 379 ms, total: 16.6 s
Wall time: 16.6 s


In [20]:
with open("./data/qa_dataset_cn_ordering.json") as json_file:
    data = json.load(json_file)

In [21]:
pprint(data[0], sort_dicts=False)

{'question': '权益购买的功能用途是什么？',
 'answer': '权益购买的功能用途是购买需要的平台版本，确定该账号所持权益，并进行付款和索取发票。这个功能可以让用户根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。同时，用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。'}


In [22]:
pprint(data[1], sort_dicts=False)

{'question': '权益购买的具体操作步骤是什么？',
 'answer': '权益购买的具体操作步骤包括登录账号后，点击【平台版本】，进入权益购买界面，【+选择权益】可进行预购版本的选择，点击下一步或者购物车，进行订单的确认和提交。订单提交后重新登录。这些步骤可以让用户方便地选择需要的平台版本，并完成订单的提交。'}


In [33]:
# data = load_dataset("json", data_files="./data/qa_dataset.json")

In [23]:
import pandas as pd
data = pd.read_csv("./data/warreconn_data_cn_ordering.csv")
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [24]:
dataset[0]

{'question': '权益购买的功能用途是什么？',
 'answer': '权益购买的功能用途是购买需要的平台版本，确定该账号所持权益，并进行付款和索取发票。这个功能可以让用户根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。同时，用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。'}

In [25]:
def generate_prompt(data_point):
    return f"""
: {data_point["question"]}
: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [26]:
train_data = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [27]:
train_data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 5
})

## Training

In [28]:
OUTPUT_DIR = "experiments"

In [29]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [30]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

In [31]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [32]:
%%time
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.613500
2,1.601300
3,1.473600
4,1.220100
5,0.932700
6,0.696300
7,0.455000
8,0.256000
9,0.143400
10,0.107100


CPU times: user 3min 5s, sys: 20 s, total: 3min 25s
Wall time: 3min 26s


TrainOutput(global_step=80, training_loss=0.11792703644605354, metrics={'train_runtime': 205.7477, 'train_samples_per_second': 1.555, 'train_steps_per_second': 0.389, 'total_flos': 2054819736453120.0, 'train_loss': 0.11792703644605354, 'epoch': 64.0})

## Save the trained model

In [33]:
model.save_pretrained("llama2_cn_trained-model_ordering")

In [47]:
# from huggingface_hub import notebook_login

# notebook_login()

In [48]:
# peft_model_name = "llama2_cn_trained-model"
# model.push_to_hub(peft_model_name)

## Load Trained Model

In [34]:
# PEFT_MODEL = f"ysw96/{peft_model_name}"
PEFT_MODEL = f"./llama2_cn_trained-model_ordering"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    # trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Inference

In [35]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.0
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [36]:
DEVICE = "cuda:0"

In [37]:
%%time
prompt = f"""
: 权益购买的功能用途是什么？
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: 权益购买的功能用途是什么？
: 权益购买的功能用途是购买需要的平台版本，确定该账号所持权益，并进行付款和索取发票。这个功能可以让用户根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。同时，用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。

CPU times: user 15.5 s, sys: 58.5 ms, total: 15.6 s
Wall time: 15.6 s


In [38]:
def generate_response(question: str) -> str:
    prompt = f"""
: {question}
:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = ":"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [39]:
prompt = "权益购买的功能用途是什么？"
print(generate_response(prompt))

权益购买的功能用途是什么？
: 权益购买的功能用途是购买需要的平台版本，确定该账号所持权益，并进行付款和索取发票。这个功能可以让用户根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。同时，用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。


In [40]:
prompt = "权益购买的具体操作步骤是什么？"
print(generate_response(prompt))

权益购买的具体操作步骤是什么？
: 权益购买的具体操作步骤包括登录账号后，点击【平台版本】，进入权益购买界面，【+选择权益】可进行预购版本的选择，点击下一步或者购物车，进行订单的确认和提交。订单提交后重新登录。这些步骤可以让用户方便地选择需要的平台版本，并完成订单的提交。同时，用户可以在订单提交时进行付款


In [41]:
%%time
prompt = "如何选择需要的平台版本？"
print(generate_response(prompt))

如何选择需要的平台版本？
: 在权益购买界面，用户可以进行预购版本的选择，点击【+选择权益】即可。用户可以根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。以下是一些可能的选择方案：

1. 权益购买的基本版本，用户可以选择这个版本，方便管理自己的平台账号和权益。这个版本提供的功能包括购买
CPU times: user 17.9 s, sys: 40.3 ms, total: 18 s
Wall time: 18 s


In [42]:
%%time
prompt = "如何确认订单并进行提交？"
print(generate_response(prompt))

如何确认订单并进行提交？
: 在权益购买界面，用户可以点击下一步或者购物车，进行订单的确认和提交。用户需要仔细核对订单信息，确保订单信息无误后再进行提交。提交订单后，用户需要重新登录账号。用户需要仔细权益购买界面，确保订单信息无误后再进行提交。提交订单后，用户需要重新登录账号。用户需要仔细权益购买界
CPU times: user 18.1 s, sys: 32.5 ms, total: 18.2 s
Wall time: 18.2 s


In [43]:
%%time
prompt = "是否可以索取发票？"
print(generate_response(prompt))

是否可以索取发票？
: 是的，权益购买功能可以进行付款和索取发票。用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。用户需要填写发票抬头、类型、寄送地址等发票信息，并确认发。用户可以选择需要的发票类型，包括平台版本、平台版本的免费版本、付款和索取发票。用户需要填写发票抬头、类型、寄送
CPU times: user 18.1 s, sys: 48 ms, total: 18.2 s
Wall time: 18.2 s


## Similar question

In [47]:
%%time
prompt = "一定要购买权益吗?"
print(generate_response(prompt)) #Q1

一定要购买权益吗?
: 是的，权益购买功能可以进行付款和索取发票。用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。用户需要填写发票抬头、类型、寄送地址等发票信息，并确认发。用户可以根据自己的需求选择不同的发票类型，以便更好地管理账务和报销。购买权益时用户需要填写发票抬头
CPU times: user 18.3 s, sys: 46.6 ms, total: 18.4 s
Wall time: 18.4 s


In [46]:
%%time
prompt = "如何购买权益?"
print(generate_response(prompt)) #Q2

如何购买权益?
: 在权益购买界面，用户可以进行付款和索取发票。用户需要填写发票抬头、类型、寄送地址等发票信息，并确认发。用户需要仔细核对发票信息，确保发票信息无误后再进行提交。提交发票后，用户需要重新登录账号。用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。用户需要�����������
CPU times: user 18.3 s, sys: 63.8 ms, total: 18.4 s
Wall time: 18.4 s


In [45]:
%%time
prompt = "如何选择不同的平台版本?"
print(generate_response(prompt)) #Q3

如何选择不同的平台版本?
: 在权益购买界面，用户可以进行预购版本的选择，点击【+选择权益】即可。用户可以根据自己的需求选择不同的平台版本，以便更好地使用平台提供的服务。以下是一些可能的选择方案：

1. 权益购买的具体操作步骤是什么？

权益购买的具体操作步骤包括登录账号后，点击【平台版本
CPU times: user 18.1 s, sys: 41.4 ms, total: 18.2 s
Wall time: 18.2 s


In [48]:
%%time
prompt = "订单在哪里查看?"
print(generate_response(prompt)) #Q4

订单在哪里查看?
: 订单可以在权益购买界面进行查看，用户可以点击【平台版本】，进入订单界面，【+选择权益】可进行订单的选择，订单信息可进行预购版本的选择，点击下一步或者购物车，进行订单的确认和提交。订单提交后重新登录。用户可以在订单界面进行付款和索取发票。用户需要填写发票抬头、类型、寄送地址等发票信息。用
CPU times: user 21.4 s, sys: 40 ms, total: 21.4 s
Wall time: 21.5 s


In [44]:
%%time
prompt = "可以开发票吗?"
print(generate_response(prompt)) #Q5

可以开发票吗?
: 是的，权益购买功能可以进行付款和索取发票。用户可以在购买权益时进行付款和索取发票，方便管理账务和报销。用户需要填写发票抬头、类型、寄送地址等发票信息，并确认发。用户可以选择需要的发票类型，如平台版本发票、权益购买发票等。用户需要填写发票信息的具体操作步骤可以参�
CPU times: user 18.1 s, sys: 35 ms, total: 18.2 s
Wall time: 18.2 s
